In [116]:
import pickle
import sentencepiece as spm
import torchdata.datapipes as dp
import torchtext.transforms as T

from torchdata.datapipes.iter import IterableWrapper, FileOpener
sp = spm.SentencePieceProcessor(model_file='spm_openstack.model')
vocabs = [sp.id_to_piece(id) for id in range(sp.get_piece_size())]
data_pipe = IterableWrapper(['openstack_train.csv'])
data_pipe = FileOpener(data_pipe, mode='r')
data_pipe = data_pipe.parse_csv(skip_lines=1, )
for sample in data_pipe:
    print(sample)
    break

['0', 'add lun# to provider_location in nexenta driver. fixes bug [ISSUEID]. let us nexenta driver work with any iscsi_helper. change-id: [CHANGEID]', '0']


In [117]:
vocabs

['<unk>',
 '<s>',
 '</s>',
 '<pad>',
 '[NUM]',
 '[URL]',
 '[COMMITID]',
 '[CHANGEID]',
 '[ISSUEID]',
 '▁',
 '.',
 '▁the',
 '-',
 '_',
 '▁to',
 ':',
 '▁change',
 'id',
 ',',
 's',
 '▁in',
 '▁a',
 '▁is',
 '▁this',
 '▁for',
 '▁of',
 '▁and',
 '▁bug',
 '▁not',
 '▁fixes',
 '▁that',
 '▁it',
 '/',
 '▁add',
 '▁be',
 'ing',
 "'",
 '▁with',
 '▁from',
 '▁on',
 '▁fix',
 '▁instance',
 '▁when',
 '▁api',
 '▁as',
 '▁an',
 '▁use',
 '▁will',
 '▁nova',
 '▁we',
 '▁if',
 '▁test',
 '()',
 '▁(',
 '▁are',
 ')',
 '▁#',
 '▁was',
 '▁tests',
 '▁*',
 "▁'",
 '▁by',
 '▁patch',
 '▁blueprint',
 '▁make',
 '▁which',
 '▁so',
 '▁volume',
 '▁code',
 'api',
 '▁image',
 '▁also',
 '▁v',
 '▁driver',
 '▁db',
 '▁new',
 '▁get',
 '▁remove',
 '▁can',
 '▁method',
 'py',
 '▁compute',
 '▁"',
 '"',
 'ed',
 '▁all',
 '▁support',
 '▁should',
 '▁exception',
 '▁set',
 '▁only',
 '▁error',
 '▁no',
 '▁have',
 '▁used',
 '▁using',
 '▁instead',
 '▁but',
 '▁or',
 '▁update',
 '▁some',
 '▁call',
 '▁do',
 '▁implements',
 '▁does',
 '▁network',
 '▁there

In [118]:
def removeAttribution(row):
    """
    Function to keep the first two elements in a tuple
    """
    return row[1:]
data_pipe = data_pipe.map(removeAttribution)
for sample in data_pipe:
    print(sample)
    break

['add lun# to provider_location in nexenta driver. fixes bug [ISSUEID]. let us nexenta driver work with any iscsi_helper. change-id: [CHANGEID]', '0']


In [119]:
from torchtext.vocab import vocab
def load_vocab(vocab_list):
    # Create a TorchText Vocab object
    vocab = vocab(vocab_list)
    vocab.set_itos(vocab_list)
    vocab.set_stoi({token: i for i, token in enumerate(vocab_list)})

    return vocab

In [120]:
from torchtext.vocab import vocab
vocabs2 = {token:i for i, token in  enumerate(vocabs)}
print(vocabs2)
vocabs3 = vocab(
    ordered_dict=vocabs2,
    specials=['<unk>','<s>','</s>','<pad>','[NUM]','[URL]','[COMMITID]','[CHANGEID]','[ISSUEID]'],
    special_first=True
)
print(vocabs3.get_itos())

{'<unk>': 0, '<s>': 1, '</s>': 2, '<pad>': 3, '[NUM]': 4, '[URL]': 5, '[COMMITID]': 6, '[CHANGEID]': 7, '[ISSUEID]': 8, '▁': 9, '.': 10, '▁the': 11, '-': 12, '_': 13, '▁to': 14, ':': 15, '▁change': 16, 'id': 17, ',': 18, 's': 19, '▁in': 20, '▁a': 21, '▁is': 22, '▁this': 23, '▁for': 24, '▁of': 25, '▁and': 26, '▁bug': 27, '▁not': 28, '▁fixes': 29, '▁that': 30, '▁it': 31, '/': 32, '▁add': 33, '▁be': 34, 'ing': 35, "'": 36, '▁with': 37, '▁from': 38, '▁on': 39, '▁fix': 40, '▁instance': 41, '▁when': 42, '▁api': 43, '▁as': 44, '▁an': 45, '▁use': 46, '▁will': 47, '▁nova': 48, '▁we': 49, '▁if': 50, '▁test': 51, '()': 52, '▁(': 53, '▁are': 54, ')': 55, '▁#': 56, '▁was': 57, '▁tests': 58, '▁*': 59, "▁'": 60, '▁by': 61, '▁patch': 62, '▁blueprint': 63, '▁make': 64, '▁which': 65, '▁so': 66, '▁volume': 67, '▁code': 68, 'api': 69, '▁image': 70, '▁also': 71, '▁v': 72, '▁driver': 73, '▁db': 74, '▁new': 75, '▁get': 76, '▁remove': 77, '▁can': 78, '▁method': 79, 'py': 80, '▁compute': 81, '▁"': 82, '"': 83,

In [121]:
def getTransform(vocabs):
    """
    Create transforms based on given vocabulary. The returned transform is applied to sequence
    of tokens.
    """
    text_tranform = T.Sequential(
        T.SentencePieceTokenizer(sp_model_path='spm_openstack.model'),
        ## converts the sentences to indices based on given vocabulary
        T.VocabTransform(vocabs),
        ## Add <sos> at beginning of each sentence. 1 because the index for <sos> in vocabulary is
        # 1 as seen in previous section
        # T.AddToken(1, begin=True),
        ## Add <eos> at beginning of each sentence. 2 because the index for <eos> in vocabulary is
        # 2 as seen in previous section
        # T.AddToken(2, begin=False)
    )
    return text_tranform
def applyTransform(sequence_pair):
    """
    Apply transforms to sequence of tokens in a sequence pair
    """

    return (
        getTransform(vocabs3)(sequence_pair[0]),
        sequence_pair[1],
    )
    


In [122]:
temp_list = list(data_pipe)
some_sentence = temp_list[798][0]
print("Some sentence=", end="")
print(some_sentence)
transformed_sentence = getTransform(vocabs3)(some_sentence)
print("Transformed sentence=", end="")
print(transformed_sentence)
index_to_string = vocabs3.get_itos()
for index in transformed_sentence:
    print(index_to_string[index], end=" ")

Some sentence=use nose and openstack nose plugin. it is a little silly since we are working on removing nose for testr, but it turns out that the old test run wrapper was in use here, which was causing all sorts of havoc. change-id: [CHANGEID]
Transformed sentence=[46, 2066, 26, 221, 2066, 124, 10, 31, 22, 21, 1636, 4038, 154, 49, 54, 1194, 39, 683, 35, 2066, 24, 2253, 18, 97, 31, 9, 2734, 156, 30, 11, 401, 51, 210, 1198, 57, 20, 46, 542, 18, 65, 57, 899, 85, 1381, 19, 25, 817, 752, 316, 10, 16, 12, 17, 15, 9, 7]
▁use ▁nose ▁and ▁openstack ▁nose ▁plugin . ▁it ▁is ▁a ▁little ▁silly ▁since ▁we ▁are ▁working ▁on ▁remov ing ▁nose ▁for ▁testr , ▁but ▁it ▁ turns ▁out ▁that ▁the ▁old ▁test ▁run ▁wrapper ▁was ▁in ▁use ▁here , ▁which ▁was ▁causing ▁all ▁sort s ▁of ▁hav o c . ▁change - id : ▁ [CHANGEID] 

In [123]:
data_pipe = data_pipe.map(applyTransform)
temp_list = list(data_pipe)
print(temp_list[0])

([33, 1660, 918, 14, 920, 13, 1019, 20, 1591, 73, 10, 29, 27, 9, 8, 10, 697, 452, 1591, 73, 180, 37, 200, 432, 13, 947, 10, 16, 12, 17, 15, 9, 7], '0')
